## Expressions 

We will develop a data structure to represent arithmetic expressions containing variables, such as $3 + 4$ or $2 + x * (1 - y)$.  

What is an expression?  An expression consists of one of these: 


1. A number
2. A variable
3. If $e_1$ and $e_2$ are expressions, then $e_1 + e_2$, $e_1 - e_2$, $e_1 * e_2$, and $e_1 / e_2$ are also expressions. 

Formally, the set of expressions is the _least_ set constructed according to the rules above. 

Thus, an expression can be either a constant, representing numbers and variables, or a composite expression, consisting of an operator, a left expression, and a right expression.  


There are (at least) two ways of representing expressions. The simplest way is to represent expressions as trees, and define operations on them. 
The more sophisticated way consists in representing expressions via classes: there will be one class for variable and constants, and one class representing composite expressions; both of these classes will be subclasses of a generic "expression" class. 

In this chapter, we will represent expression as trees, to gain experience with writing recursive functions on trees; in the next chapter, we will show how to represent them more elegantly as classes.

We will represent expressions as trees.  A number will be represented via a number; a variable via a string, and the expression $e_1 \odot e_2$ via the tuple $(\odot, e_1, e_2)$, for $\odot \in \{+, -, *, / \}$.

For example, we will represent $2 * (x + 1)$ via:

    ('*', 2, ('+', 'x', 1))

In [ ]:
e = ('*', 2, ('+', 'x', 1))


In particular, we will consider expressions built out of the four arithmetic operators `"+"`, `"-"`, `"*"`, `"/"`. 

### A compute function

Let us define a function compute() that takes one such expression, and returns the expression obtained by performing all possible numerical computation.
We consider first the simple case of an expression where the only operators that can appear are `+` and `-`, and where there are no variables.  

Let us create an exception to raise when we cannot interpret an expression. 

In [ ]:
class IllegalExpression(Exception):
    pass


In [ ]:
def compute_plus_minus(e):
    if isinstance(e, tuple):
        # We have an expression.
        op, l, r = e
        # We compute the subexpressions.
        ll = compute_plus_minus(l)
        rr = compute_plus_minus(r)
        # And on the basis of those, the whole expression.
        if op == "+":
            return ll + rr
        elif op == "-":
            return ll - rr
        else:
            raise IllegalExpression(repr(e))
    else:
        # base expression; just return the number.
        return e


In [ ]:
compute_plus_minus(("+", 4, 5))


9

In [ ]:
compute_plus_minus(("+", ("-", 3, 1), ("+", 4, 9)))


15

We can try the exception: 

In [ ]:
import traceback

try:
    compute_plus_minus(("^", 3, 4))
except:
    traceback.print_exc()


Traceback (most recent call last):
  File "<ipython-input-7-81794c941225>", line 4, in <module>
    compute_plus_minus(("^", 3, 4))
  File "<ipython-input-4-02bebdddfcd8>", line 14, in compute_plus_minus
    raise IllegalExpression(repr(e))
IllegalExpression: ('^', 3, 4)


Let us define a helper function `calc`, which takes as argument an operator and two numbers, and computes the required operation.  It will make it easier to write the rest of the code. 

In [ ]:
def calc(op, left, right):
    if op == "+":
        return left + right
    elif op == "-":
        return left - right
    elif op == "*":
        return left * right
    elif op == "/":
        return left / right


## Question 1

Extend the `compute_plus_minus` to a `compute_arithmetic` function that can deal with all four arithmetic operators. You may use the above `calc` helper if you wish. 

In [ ]:
def compute_arithmetic(e):
    """Computes the value of an arithmetic expression e."""
    ### YOUR CODE HERE
    if isinstance(e,tuple):
        op , l , r = e
        ll = compute_arithmetic(l)
        rr = compute_arithmetic(r)
        return calc(op,ll,rr)
    else:
        return e

In [ ]:
## Simple tests for one-level expressions. 2 points. 

assert compute_arithmetic(3) == 3
assert compute_arithmetic(("+", 3, 5)) == 8
assert compute_arithmetic(("-", 3, 5)) == -2
assert compute_arithmetic(("*", 3, 5.5)) == 16.5
assert compute_arithmetic(("/", 10, 5)) == 2

In [ ]:
## Tests for multilevel expressions.  3 points. 

e = ("-", ("+", 3, 4), ("*", 5, 3))
assert compute_arithmetic(e) == -8

e = ("*", ("/", 8, 4), ("*", 5, 3))
assert compute_arithmetic(e) == 30

e = ("*", ("/", 8, 4), ("*", 5, 3.2))
assert compute_arithmetic(e) == 32

e = ("*", ("/", 8, 4), ("*", ("-", 9, 6), 5))
assert compute_arithmetic(e) == 30

In [ ]:
## Hidden tests for expressions.  5 points. 


## Expressions with variables

If an expression can have variables, we can distinguish three types of expressions: 

* Numbers 
* Variables
* Composite expressions. 

To facilitate writing code, let us define for you three helper functions that tell us the type of an expression. 

In [ ]:
from numbers import Number # The mother class of all numbers.

def isnumber(e):
    return isinstance(e, Number)

def isvariable(e):
    return isinstance(e, str)

def iscomposite(e):
    return isinstance(e, tuple)


The idea we use to simplify an expression is the following: 

* If the expression is a Number, you return a number: it's already simplified. 
* If the expression is a variable, you return the variable (that is, the expression unchanged); there is nothing to be done. 
* If the expression is an operation, such as "+", "-", ..., then you consider the right and left children, and you reason: 
    * If all the two children are numbers, then you can compute the operation and return the result. 
    * Otherwise, again, there is nothing that can be done, and you return the expression unchanged. 


In [ ]:
def simplify_once(e):
    if isnumber(e) or isvariable(e):
        # No simplification possible.
        return e
    else:
        op, l, r = e
        if isnumber(l) and isnumber(r):
            return calc(op, l, r)
        else:
            # We cannot do anything.
            return e


Let's see how this works.

In [ ]:
simplify_once(3)


3

In [ ]:
simplify_once(("+", "x", 1))


('+', 'x', 1)

Yes, there was nothing we could simplify.  Let's try now with something we can simplify: 

In [ ]:
simplify_once(('-', 5, 4))


1

Can we simplify bigger expressions? 

In [ ]:
simplify_once( ('+', 6, ('-', 7, 2)) )


('+', 6, ('-', 7, 2))

No.  It's not wrong, but nothing happens. Why?  

Because when `simplify_once` looks at the top expression, it sees a `+`.  It then asks (line 9): are both children numbers?  And the answer is No: the left child, 6, is a number, but the right child is an expression. 

We need to _first_ simplify the expressions corresponding to the children, and only _then_ ask whether the children of the node are both numbers.  In other words, _simplify_ needs to be a _recursive_ function, which for a node, first tries to simplify the children, and only then tries to simplify the whole expression. 

## Question 2

Write a version of `simplify` that is able to carry on computation on multi-level expressions, including (but not limited to) `('+', 6, ('-', 7, 2))`. 

In [ ]:
def simplify(e):
    """Simplifies an expression containing variables, carrying out all possible computations."""
    ### YOUR CODE HERE
    if isnumber(e) or isvariable(e):
        # No simplification possible.
        return e
    else:
        op, l, r = e
        ll = simplify(l)
        rr = simplify(r)
        if isnumber(ll) and isnumber(rr):
            return calc(op, ll, rr)
        else:
            return (op, ll, rr)    

In [ ]:
## Tests for simplify. 2 points. 

e = ('+', 6, ('-', 7, 2))
assert simplify(e) == 11

e = ('+', 6, ('-', "x", 2))
assert simplify(e) == e

e = ('+', "cat", ('-', 7, 2))
assert simplify(e) == ('+', "cat", 5)

e = ('*', ('+', 2, 3), ('-', 7, 2))
assert simplify(e) == 25

e = ('*', ('+', 2, 3), ('-', 7, "monkey"))
assert simplify(e) == ('*', 5, ('-', 7, 'monkey'))

In [ ]:
#my test
e = ('+', ('+', 6, 3), ('-', 7, 2))
print(simplify(e))
assert simplify(e) == 14

14


In [ ]:
## Hidden tests for simplify.  4 points. 


## Question 3: Evaluating expressions with respect to a variable valuation.

The function `simplify` we asked you to write above can perform all numerical computations, but stops whenever it encounters a variable. It cannot do any better, in fact, because it does not know the values of variables. 

If we specify values for variables, we can then use those values in the computation, replacing each variable whose value is specified with the value itself. 

A _variable valuation_ is a mapping from variables to their values; we can represent it simply as a dictionary associating to each variable a number:

In [ ]:
varval = {'x': 3, 'y': 8}


You can extend the evaluation function to take as input a variable valuation.  The idea is that, when you find a variable, you try to see whether its value is specified in the variable valuation.  If it is, you can replace the variable with the value, and carry on.  If it is not, you leave the variable as it is, since you cannot evaluate it. 

To check if a variable (a string) `s` is in a dictionary `d`, you can test

    s in d

and to get the value, in case it is present, you can just do `d[s]` of course.  We let you develop the code. 

In [ ]:
"""psudo code:
    check to see if the variable in e is in varval
    then if it is, replace the variable with the value
    If it is not, you leave the variable as it is """

'psudo code:\n    check to see if the variable in e is in varval\n    then if it is, replace the variable with the value\n    If it is not, you leave the variable as it is '

In [ ]:
### Evaluating an expression with respect to a variable evaluation
def compute(e, varval={}):
   ### YOUR CODE HERE
   ## Tests for compute.
    if (isvariable(e)) and (e in varval):
        e = varval[e]
    if isnumber(e) or isvariable(e):
        return e
    else:
        op, l, r = e
        ll = compute(l, varval)
        rr = compute(r, varval)
        if isnumber(ll) and isnumber(rr):
            return calc(op, ll, rr)
        else:
            return (op, ll, rr)    

In [ ]:
## Tests for compute.  4 points. 

e = ('*', 2, ('+', 'x', ('-', 3, 2)))
assert compute(e) == ('*', 2, ('+', 'x', 1))
assert compute(e, varval={'x': 6}) == 14
assert compute(e, varval={'y': 10}) == ('*', 2, ('+', 'x', 1))

e = ('+', ('-', 'yy', 3), ('*', 'x', 4))
assert compute(e, varval={'x': 2}) == ('+', ('-', 'yy', 3), 8)
assert compute(e, varval={'yy': 3}) == ('+', 0, ('*', 'x', 4))
assert compute(e, varval={'x': 2, 'yy': 3}) == 8

In [ ]:
### Hidden tests for compute.  6 points. 


If we provide the values for only some of the variables, the compute function defined above, will plug in the values for those variables and perform all computations possible.  Of course, if the expression contains variables for which the valuation does not specify a value, the resulting expression will still contain those variables: it will not be simply a number.  In computer science, evaluating an expression as far as possible using the values for a subset of the variables is knwon as _partial evaluation_.

## When are two expressions equal? 

> _Or: it's better to be lucky than to be smart._

> _Or: if you don't know how to do it right, do it at random._ 

> _Or: the power of randomization._

We now consider the following problem: given two expressions $e$ and $f$, how can we decide whether they are equal in value, that is, whether they yield always the same value for all values of the variables? 

This _"value equality"_ is a different notion from the structural equality we defined before.  For instance, the two expressions `("+", "x", 1)` and `("-", (*, 2, "x"), "x")` are not structurally equal, but they are equal in values.  

How can we test for value equality of expressions?  There are two ways: the high road, and the pirate road. Of course, we take the pirate road.   

The high-road approach consists in trying to demonstrate, in some way, that the two expressions are equal.  One way of doing so would be to define a set of [rewriting rules](https://en.wikipedia.org/wiki/Rewriting) for expressions, that try to transform one expression into the other; this would mimick the process often done by hand to show that two expressions are equal.  Another way would be to use theorem provers that can reason about expressions and real numbers, such as [PVS](https://pvs.csl.sri.com).  The problem is that these approaches are a lot of work.  Is there a way to be lazy, and still get the job done? 

There is, it turns out.  Suppose you have two expressions $f, g$ containing variable $x$ only.  The idea is that if $f$ and $g$ are built with the usual operators of algebra, it is exceedingly unlikely for $f$ and $g$ to give the same value  many values of $x$, and yet not be always equal.  This would not be true if our expressions could contain if-then-else statements, but for the operators we defined so far, it holds.  Indeed, one could be more precise, and try to come up with a theorem of the form: 

> If $f$ and $g$ have "zerosity" $n$, and are equal for $n+1$ values of $x$, then they are equal for all values of $x$. 

We could then try to define the "zerosity" of an expression to make this hold: for example, for two polynomials of degree at most $d$, once you show that they are equal for $d+1$ points, they must be equal everywhere ([why?](https://en.wikipedia.org/wiki/Fundamental_theorem_of_algebra)).  But this again would be a smart approach, and we are trying to see if we can solve the problem while being as stupid as possible.  So our idea will simply be: pick 1000 values of $x$ at random; if the two expressions are equal for all the values, then they must be equal everywhere.  This is a somewhat special case of a [Monte Carlo method](https://en.wikipedia.org/wiki/Monte_Carlo_method), a method used to estimate the probability of complex phenomena (where expression equality is our phenomenon).

There are only two wrinkles with this.  The first is that an expression can contain many variables, and we have to try to value assignments for all of the variables.  This is easy to overcome; we just need some helper function that gives us the set of variables in a function.  The second wrinkle is: how do we generate the possible value assignments?  How big do these values need to be on average?  According to what probability distribution?  We could dive into a lot of theory and reasoning about how to compute appropriate probability distributions, but since our goal is to be stupid, we will use one of the simplest distributions with infinite domain: the Gaussian one. 

Let us start by writing the function `variables` such that, if `e` is an expression, `variables(e)` is the set of variables that appear in it.

## Question 4: define `variables`

In [ ]:
"""psudo code:
    copy three
    1st: keep the last else and get rid of if 
    2nd: write two if statements: one: isvariable(e) return {e}
    if isnumber(e) return empty a set () 
    under the else statment return union between recurssions """


'psudo code:\n    copy three\n    1st: keep the last else and get rid of if \n    2nd: write two if statements: one: isvariable(e) return {e}\n    if isnumber(e) return empty a set () \n    under the else statment return union between recurssions '

In [ ]:

### Exercise: define `variables`
### YOUR CODE HERE
def variables(e, varval={}):
    if isvariable(e):
        return {e}
    if isnumber(e):
        return set()
    else:
        op, l, r = e
        ll = variables(l, varval)
        rr = variables(r, varval)
        if isnumber(ll) and isnumber(rr):
            return calc(op, ll, rr)
        else:
            return ll.union(rr)


In [ ]:
### Tests for `variables`. 2 points. 

e = ('*', ('+', 'x', 2), ('/', 'x', 'yay'))
assert variables(e) == {'x', 'yay'}

e = ('-', ('+', 'a', 2), ('*', 'c', 'c'))
assert variables(e) == {'a', 'c'}


In [ ]:
### Hidden tests for variables. 4 points. 


## Question 5: value equality

We ask you to write the `value_equality` method.  
Given the two expressions $e$ and $f$, first compute the set of variables $V$ that appear in either $e$ or $f$. 
Then, the idea consists in performing `num_sample` times the following test for equality: 

* First, produce a variable assignment (a dictionary) mapping each variable in $V$ to a random value.  Choose these random values from the gaussian distribution centered around 0 and with standard deviation 10 (for instance; any continuous distribution with infinite domain would work).  You can obtain such numbers using [`random.gauss(0, 10)`](https://docs.python.org/3/library/random.html#random.gauss). 
* Then, compute the values of $e$ and $f$ with respect to that variable evaluation.  If the values are closer than a specified tolerance `tolerance`, you consider $e$ and $f$ equal (for that variable valuation).  Otherwise, you can stop and return that $e$ and $f$ are different. 

If you can repeat the process `num_sample` times, and $e$ and $f$ are considered equal every time, then you declare them equal. 

In [ ]:
"""Q5 Guide:

#Outside Loop:
Loop through num samples
    #Inside Outer Loop:
    Get the combined set of variables for e and f using .union()
    Initialize dictionary 
    #Inner Loop
    Loop through the combined set of variables and initialize each ...
        dictionary[looped elem] to random.gauss(0, 10)
    #End Inner Loop
    #1 Call compute() using e and dictionary initialized before
    #2 Call compute() using f and dictionary initialized before
    Check if abs(#1 - #2) > tolerance and return False if so
    #End Inside Outer loop
#Outside Loop
Return True
#End Outside Loop"""

'Q5 Guide:\n\n#Outside Loop:\nLoop through num samples\n    #Inside Outer Loop:\n    Get the combined set of variables for e and f using .union()\n    Initialize dictionary \n    #Inner Loop\n    Loop through the combined set of variables and initialize each ...\n        dictionary[looped elem] to random.gauss(0, 10)\n    #End Inner Loop\n    #1 Call compute() using e and dictionary initialized before\n    #2 Call compute() using f and dictionary initialized before\n    Check if abs(#1 - #2) > tolerance and return False if so\n    #End Inside Outer loop\n#Outside Loop\nReturn True\n#End Outside Loop'

In [ ]:
### Exercise: implementation of value equality

import random

def value_equality(e, f, num_samples=1000, tolerance=1e-6):
    """Return True if the two expressions self and other are numerically
    equivalent.  Equivalence is tested by generating
    num_samples assignments, and checking that equality holds
    for all of them.  Equality is checked up to tolerance, that is,
    the values of the two expressions have to be closer than tolerance.
    It can be done in less than 10 lines of code."""
    ### YOUR CODE HERE

    for _ in range(num_samples):
      combo = variables(e).union(variables(f))
      d = {}
      for var in combo:
        d[var] = random.gauss(0, 10)

      val1 = compute(e, d)
      val2 = compute(f, d)
      if abs(val1 - val2) > tolerance:
        return False
        
    return True


In [ ]:
### Tests for value equality. 4 points. 

e1 = ('+', ('*', 'x', 1), ('*', 'y', 0))
e2 = 'x'
assert value_equality(e1, e2)

e3 = ('/', ('*', 'x', 'x'), ('*', 'x', 1))
assert value_equality(e1, e3)

e4 = ('/', 'y', 2)
assert not value_equality(e1, e4)
assert not value_equality(e3, e4)
assert not value_equality(e4, e3)

e5 = ("+", "cat", ("-", "dog", "dog"))
assert value_equality(e5, "cat")
assert value_equality("cat", e5)

e6 = ("-", "hello", "hello")
assert value_equality(e6, 0)
assert value_equality(0, e6)

In [ ]:
### Hidden tests for value equality.  6 points. 


## Symbolic Expressions

The notation we developed enables the representation of _symbolic expressions:_ expressions in which not only numbers appear, but also symbols.  Accordingly, we can perform _symbolic_ operations on the expressions: operations that encode what you do with pencil and paper when you work on an expression. 

Our first symbolic operations will be the implementation of _derivatives._

### Derivatives

Given an expression $f$ and a variable $x$, we can compute symbolically the (partial) derivative

$$
\frac{\partial f}{\partial x}
$$

of $f$ with respect to $x$. 
For instance:

$$
\begin{align*}
\frac{\partial}{\partial x}(x^2 + 3y + 4) & = 2x \\[1ex]
\frac{\partial}{\partial x}(x^2y + xy^2 + z) & = 2yx + y^2 + z \; . 
\end{align*}
$$

Here, the "partial" in partial derivative simply means: if you need to take the derivative with respect to a specific variable, simply treat all other variables as constants. 

Computing symbolic derivatives seems complicated, but we can take it in steps.  Our expression trees have leaves, consisting of 

* numerical constants
* variables

and operators, corresponding to `+`, `-`, `*`, `/`. 
Therefore, we break down the task into the computation of the symbolic derivative for numerical constants, variables, and arithmetic operators.  The cases for the leaf nodes are:  

* For a constant $c$, $\partial c / \partial x = 0$.
* For a variable $y \neq x$,  $\partial y / \partial x = 0$.
* $\partial x / \partial x = 1$.

For operators, we can use:

$$
 \begin{align*}
 \frac{\partial}{\partial x}(f \pm g) & = \frac{\partial f}{\partial x} \pm \frac  {\partial g}{\partial x}, \\[1ex]
 \frac{\partial}{\partial x}(f \cdot g) & = \frac{\partial f}{\partial x} \cdot g + f \cdot \frac{\partial g}{\partial x}, \\[1ex]
 \frac{\partial}{\partial x}\left(\frac{f}{g}\right) & = \frac{\frac
  {\partial f}{\partial x} \cdot g - f \cdot \frac{\partial g}{\partial x}}{g^2}. 
\end{align*}
$$

## Question 6: derivative of a leaf expression

Let's start from a leaf expression.  The function `derivate_leaf` takes as argument an expression that is a leaf, and a variable, and returns the symbolic derivative of the leaf writh respect to the variable. 

In [ ]:
### Derivation of a leaf expression

def derivate_leaf(e, x):
    """This function takes as input an expreession e and a variable x,
    and returns the symbolic derivative of e wrt. x, as an expreession."""
    if e == x:
        return 1  
    else:
        return 0

In [ ]:
## Derivative of a leaf expression. 2 points. 

assert derivate_leaf("x", "x") == 1
assert derivate_leaf("x", "y") == 0
assert derivate_leaf("y", "z") == 0
assert derivate_leaf(4, "x") == 0


In [ ]:
## Hidden tests for derivative of a leaf expression. 3 points. 


## Question 7: Implementation of derivative

We now let you implement the derivative of a general expression.  The function is recursive: for instance, if the expression is (`*`, $f$, $g$), to compute its derivative with respect to a variable $x$, we use: 

$$
\frac{\partial}{\partial x}(f \cdot g) = \frac{\partial f}{\partial x} \cdot g + f \cdot \frac{\partial g}{\partial x}
$$

and so we need to recursively call symbolic derivation on the $f$ and $g$ sub-expressions, to obtain $\partial f / \partial x$ and $\partial g / \partial x$, and then produce and return an expression representing the result. 

**Important:** The code for checking expression equality is _not_ able to cope with the commutativity of `+` and `*`.  So, in your solution, please use _exactly_ these forms: 

$$
 \begin{align*}
 \frac{\partial}{\partial x}(f \pm g) & = \frac{\partial f}{\partial x} \pm \frac  {\partial g}{\partial x}, \\[1ex]
 \frac{\partial}{\partial x}(f \cdot g) & = \frac{\partial f}{\partial x} \cdot g + f \cdot \frac{\partial g}{\partial x}, \\[1ex]
 \frac{\partial}{\partial x}\left(\frac{f}{g}\right) & = \frac{\frac
  {\partial f}{\partial x} \cdot g - f \cdot \frac{\partial g}{\partial x}}{g^2}. 
\end{align*}
$$

and not, for instance, $\frac{\partial}{\partial x}(f \cdot g) = g \cdot \frac{\partial f}{\partial x} + f \cdot \frac{\partial g}{\partial x}$, which is mathematically equivalent, but would be considered different by the tests. 


In [ ]:
### Implement `derivate`

def derivate(e, x):
    if type(e) != type(tuple()):
        if e == x:
            return 1  
        else:
            return 0
    else:
        op, l, r = e
        if op == '*':
            return ('+', ('*', derivate(l, x), r), ('*', l, derivate(r, x)))
        elif op == '+' or op == '-':
             return (op, derivate(l, x), derivate(r, x))
        else:
            return ('/', ('-', ('*', derivate(l, x), r), ('*', l, derivate(r, x))),('*', r, r))

In [ ]:
### Tests for `derivate` for single-operator expressions. 4 points. 

assert derivate(('+', 'x', 'x'), 'x') == ('+', 1, 1)
assert derivate(('-', 4, 'x'), 'x') == ('-', 0, 1)
assert derivate(('*', 2, 'x'), 'x') == ('+', ('*', 0, 'x'), ('*', 2, 1))
assert derivate(('/', 2, 'x'), 'x') == ('/', ('-', ('*', 0, 'x'), ('*', 2, 1)), ('*', 'x', 'x'))


In [ ]:
### Hidden tests for `derivate` for single-operator expressions. 6 points.


In [ ]:
### Tests for `derivate` for composite expressions. 3 points. 

e1 = ('*', 'x', 'x')
e2 = ('*', 3, 'x')
num = ('-', e1, e2)
e3 = ('*', 'a', 'x')
den = ('+', e1, e3)
e = ('/', num, den)

f = ('/',
 ('-',
  ('*',
   ('-',
    ('+', ('*', 1, 'x'), ('*', 'x', 1)),
    ('+', ('*', 0, 'x'), ('*', 3, 1))),
   ('+', ('*', 'x', 'x'), ('*', 'a', 'x'))),
  ('*',
   ('-', ('*', 'x', 'x'), ('*', 3, 'x')),
   ('+',
    ('+', ('*', 1, 'x'), ('*', 'x', 1)),
    ('+', ('*', 0, 'x'), ('*', 'a', 1))))),
 ('*',
  ('+', ('*', 'x', 'x'), ('*', 'a', 'x')),
  ('+', ('*', 'x', 'x'), ('*', 'a', 'x'))))

assert derivate(e, 'x') == f

In [ ]:
### Hidden tests for `derivate` for composite expressions. 7 points. 


### Testing the derivative via its definition

One of the best ways of testing a solution to a difficult problem consists in implementing a different solution, and then comparing the two. 

To help test your implementation of derivative, we can use the following property of derivative: 

$$
\frac{\partial f}{\partial x} \approx \frac{f(x + \Delta) - f(x)}{\Delta}
$$




In [ ]:
def derivate_approx(f, x, varval, delta=0.0001):
    """Computes the derivative of f with respect to x, for a given delta,
    using the (f(x + delta) - f(x)) / delta method. """
    # This is f(x)
    f_x = compute(f, varval=varval)

    varval_delta = dict(varval)
    varval_delta[x] += delta
    # This is f(x + delta)
    f_x_plus_delta = compute(f, varval=varval_delta)

    return (f_x_plus_delta - f_x) / delta


In [ ]:
# This is x^2 + 3x.  Its derivative is 2x + 3.
f = ("+", ("*", "x", "x"), ("*", "x", 3))
# The derivative, at x= 2, should be close to 7.
derivate_approx(f, "x", varval=dict(x=2))


7.000100000027487

On the basis of this idea, write a function `test_derivative` that takes as arguments: 

* an expression `f`
* its symbolic derivative expression `df` computed with respect to variable `x`
* the variable `x`
* a `delta` to compute the derivative of `f` with respect to `x` using `derivate_approx`,
* a `tolerance` within which to consider the solution correct.
* a `num_trials` specifying how many times to perform the validation. 

For each validation, one must first choose a random variable valuation for the variables appearing in `f`, as done for the expression equality check.  For this variable valuation, one then computes: 

* the value of the symbolic derivative `df`
* the value of the experimental derivative of `f`, computed using `derivate_approx`

and one checks that the two values are closer, in absolute value, than `tolerance`.  If any of the `num_trials` checks fail, we return `False`, to indicate that `df` is not the symbolic derivative of `f`.  If all `num_trial` checks pass, then we return `True`. 



To help clarify what you have to do, let's do it by hand once.  Consider this expression: 

In [ ]:
f = ("+", ("*", "cat", "cat"), ("*", "dog", "cat"))


We are going to consider the symbolic derivative with respect to `cat`.

In [ ]:
x = "cat" # Variable wrt which we derivate.
# This is our first candidate at symbolic derivative.
df1 = ("+", ("*", 2, "cat"), "dog")
# And this is our second attempt.
df2 = ("+", ("*", 2, "cat"), ("*", "dog", "cat"))

# Let us come up with a random variable valuation. Well, not random actually.
valu = {"cat": 1.2, "dog": 3.34}

# Let us compute the values of df1 and df2, and of the approximation of the derivative.
print("df1:", compute(df1, varval=valu))
print("df2:", compute(df2, varval=valu))
print("approximate:", derivate_approx(f, x, valu))


df1: 5.74
df2: 6.4079999999999995
approximate: 5.740099999993475


From this we see that `df2` is not the symbolic derivative of `f`, as the error is quite large (6.4 vs. 5.74, approximately). 

## Question 8: implement `test_derivative`

Now you have to code `test_derivative`, to perform this test automatically.  We give you the function `similar`, which you should use to test whether two numbers `x`, `y` are similar within `epsilon`, via `similar(x, y, epsilon)`. According to this function, two positive numbers $x, y$ are similar within a given $\epsilon$ if either 

$$
|x-y| < \epsilon
$$ 

or 

$$
\frac{\max(x,y)}{\min(x,y) + \epsilon} < 1 + \epsilon \; . 
$$

This implements a robust test for equality that is necessary here. 

In [ ]:
def similar(x, y, epsilon):
    if x < 0 and y < 0:
        # If they are negative, max and min play opposite roles. 
        return similar(-x, -y, epsilon)
    if abs(x - y) < epsilon:
        return True
    else:
        return max(x, y) / (min(x, y) + epsilon) < 1 + epsilon

When you obtain the symbolic derivative value `sym_d` and the approximate derivative value `approx_d`, compare them via 

    similar(sym_d, approx_d, epsilon)

In [ ]:
### Implementation of `test_derivative`

def test_derivative(f, df, x, delta=0.0001, epsilon=0.1, num_tests=1000):
    """See above."""
    ### YOUR CODE HERE
    for _ in range(num_tests):
        combo = variables(f).union(variables(df))

        d = {}
        for var in combo:
            d[var] = random.gauss(0, 10)
        if x not in d:
            if df == 0:
                return True
            else:
                return False

        val1 = derivate_approx(f, x, d, delta)
        val2 = compute(df, d)
        
        if not similar(val1, val2, epsilon):
            return False

    return True

In [ ]:
### Tests for test_derivative, 4 points. 

f = ("+", ("*", "cat", "cat"), ("*", "dog", "cat"))
df1 = ("+", ("*", 2, "cat"), "dog")
df2 = ("+", ("*", 2, "cat"), ("*", "dog", "cat"))

assert test_derivative(f, df1, "cat")
assert not test_derivative(f, df2, "cat")
assert not test_derivative(f, df1, "dog")
assert not test_derivative(f, df1, "donkey")
assert test_derivative(f, 0, "donkey")


In [ ]:
### Hidden tests for test_derivative, 6 points. 


## Challenge Question
## Question 9: Distributive property

If you truly want to test your teeth on a difficult recursion, here is grist for your teeth. 
This question is worth just a few points, so that students can score well even without doing it. 
But if you are starving for a difficult question, here is one. 

**This is difficult.  Attempt at your own risk.  Instructor is not responsible for the many hours you may end up spending on this problem. Please, do not ask for help about this question in office hours; let people who are doing the regular questions benefit from office hour time.**

Implement the `apply_distributive_property` function, which applies the distributive property to an expression to push down multiplication to the leaves as far as possible.  The function should apply, recursively, the following transformations: 

$$
f (g \pm h) \Rightarrow fg \pm fh \qquad
(f \pm g) h \Rightarrow fh \pm gh
$$

until the multiplications cannot be moved down further in the expression tree.

_Hint:_ If you have $e = e_1 \odot e_2$, you first must apply distributivity to $e_1$ and $e_2$, obtaining $e'_1$ and $e'_2$.  If $\odot \neq *$, then you can just return $e'_1 \odot e'_2$. If $\odot = *$, then you must consider whether $e'_1$ or $e'_2$ has the form $g_1 \pm g_2$.  Assume $e'_2$ does.  This means your expression is $e'_1 * (g_1 \pm g_2)$, and you need to apply distributivity:

$$
e'_1 * (g_1 \pm g_2) \Rightarrow (e'_1 * g_1) \pm (e'_1 * g_2)
$$

The trick is that _after_ you apply distributivity and obtain $(e'_1 * g_1) \pm (e'_1 * g_2)$, you must _again_ apply distributivity to both

$$
(e'_1 * g_1), \qquad (e'_1 * g_2)
$$

obtaining $f_1, f_2$, respectively.  This step is necessary because $(e'_1 * g_1)$ now has $*$ as top operator, and you may need to "push it down" using distributivity, obtaining $f_1$; similarly for $(e'_1 * g_2)$.   Finally, you can return $f_1 \pm f_2$. 


In [ ]:
### Exercise: Implement `apply_distributive`

def apply_distributive(e):
    """Applies the distributive property to an expression e."""
    ### YOUR CODE HERE


In [ ]:
### Simple test for distributivity. 2 points.

## Here is a definition of equality that disregards order.
def is_distributed(exp):
    def is_plusminus(exp):
        return isinstance(exp,tuple) and exp[0] in '+-'

    if isinstance(exp, tuple):
        op, e1, e2 = exp
        if op == '*' and (is_plusminus(e1) or is_plusminus(e2)):
            return False
        return is_distributed(e1) and is_distributed(e2)
    else:
        return True

import random


def applied_distributivity(f, g):
    return is_distributed(f) and value_equality(f, g)

# Simple test

e = ('*', ('+', 1, 2), ('-', 3, 4))
f = ('+', ('-', ('*', 1, 3), ('*', 1, 4)), ('-', ('*', 2, 3), ('*', 2, 4)))
assert applied_distributivity(apply_distributive(e), f)



TypeError: ignored

In [ ]:
### More complicated tests for distributivity. 2 points. 

# More complex tests

e = ('*', ('+', 1, 2), ('-', 3, 4))
e2 = ('*', e, ('+', 5, 6))
f = ('+',
 ('-',
  ('+', ('*', ('*', 1, 3), 5), ('*', ('*', 1, 3), 6)),
  ('+', ('*', ('*', 1, 4), 5), ('*', ('*', 1, 4), 6))),
 ('-',
  ('+', ('*', ('*', 2, 3), 5), ('*', ('*', 2, 3), 6)),
  ('+', ('*', ('*', 2, 4), 5), ('*', ('*', 2, 4), 6))))

assert applied_distributivity(apply_distributive(e2), f)

e = ('*', ('*', ('+', 1, 2), ('-', 3, 4)), ('*', ('-', 5, 6), ('+', 7, 8)))
f = ('+',
 ('-',
  ('-',
   ('+', ('*', ('*', 1, 3), ('*', 5, 7)), ('*', ('*', 1, 3), ('*', 5, 8))),
   ('+', ('*', ('*', 1, 3), ('*', 6, 7)), ('*', ('*', 1, 3), ('*', 6, 8)))),
  ('-',
   ('+', ('*', ('*', 1, 4), ('*', 5, 7)), ('*', ('*', 1, 4), ('*', 5, 8))),
   ('+', ('*', ('*', 1, 4), ('*', 6, 7)), ('*', ('*', 1, 4), ('*', 6, 8))))),
 ('-',
  ('-',
   ('+', ('*', ('*', 2, 3), ('*', 5, 7)), ('*', ('*', 2, 3), ('*', 5, 8))),
   ('+', ('*', ('*', 2, 3), ('*', 6, 7)), ('*', ('*', 2, 3), ('*', 6, 8)))),
  ('-',
   ('+', ('*', ('*', 2, 4), ('*', 5, 7)), ('*', ('*', 2, 4), ('*', 5, 8))),
   ('+', ('*', ('*', 2, 4), ('*', 6, 7)), ('*', ('*', 2, 4), ('*', 6, 8))))))

assert applied_distributivity(apply_distributive(e), f)

